# Revised case normalization for Hirslanden Aarau 2018

Notes:

- 17 of 17 revised DtoD cases were joined with the BFS cases from the DB.



In [3]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.py.global_configs import *
from src.revised_case_normalization.py.normalize import normalize
from src.revised_case_normalization.py.revise import revise
from src.revised_case_normalization.py.group import group
from src.revised_case_normalization.py.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI_Aarau_Birshof_ST. Anna.xlsx'),
                     'Hirslanden Aarau', '2018', 'Aarau 2018')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI_Aarau_Birshof_ST. Anna.xlsx', hospital_name_db='Hirslanden Aarau', year='2018', sheet='Aarau 2018')


In [1]:
revised_cases_df = normalize(file_info)
revised_cases_df

NameError: name 'normalize' is not defined

# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-04 11:06:17.130 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:89 - Read 10112 rows from the DB, for the hospital 'Hirslanden Aarau' in 2018


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,431229,0041855716,I7022,"[T828, Y828, I708, I1090, Z9588, ...]",395014:L:20180918,"[004C12::20180918, 395011:L:20180918, 0041::20180918, 004B18:L:20180918, 004B45:L:20180918, ...]",W,78,0,0,1,0,01,2018-09-18,0,00,2018-09-19
1,431363,0041719219,I495,"[I484, I480, I1090, I2519, I340, ...]",893909::20180317,[],M,91,0,0,4,0,01,2018-03-17,0,00,2018-03-21
2,431860,0041648887,I5014,"[I5001, I2513, I2522, N184, I1320, ...]",99B712::20180117,"[8945::20180117, 860A06:L:20180122, 948X40::20180122]",M,64,0,0,8,8,01,2018-01-17,0,00,2018-01-25
3,432553,0041817227,I350,"[I2512, I1090, E788, F171, I7101, ...]",35F121::20180924,"[361122::20180924, 361C11::20180924, 396111::20180924, 3963::20180924, 3964::20180924, ...]",M,76,0,0,9,8,01,2018-09-23,0,00,2018-10-02
4,432647,0041665678,I5001,"[J91, I5013, I7020, L304, N185, ...]",887211::20180130,"[887963:B:20180130, 887964:B:20180130, 887975:B:20180130, 3491:R:20180201]",M,82,0,0,17,0,01,2018-01-30,0,00,2018-02-16
5,433607,0041698586,I7024,"[Z9588, N184, Z922, I743]",395011:R:20180315,"[397511:R:20180315, 395021:R:20180315, 397510:R:20180315, 0042::20180315, 004B18:R:20180315, ...]",M,93,0,0,2,0,01,2018-03-15,0,00,2018-03-17
6,433746,0041820204,N390,"[A418, R650, N182, I2519, I1090, ...]",992217::20180728,[],M,84,0,0,5,0,01,2018-07-28,0,00,2018-08-02
7,433916,0041769953,N1781,"[Y579, I1100, I480, I5001, G2010, ...]",992909::20180519,[],M,70,0,0,6,0,01,2018-05-19,0,00,2018-05-25
8,434365,0041827376,S2206,"[X599, E559, G2000, E538, D689]",816610::20180814,"[889714::20180810, 883840::20180810, 774920::20180814, 948X40::20180816]",W,78,0,0,13,0,01,2018-08-09,0,00,2018-08-22
9,435248,0041803251,I208,"[I2513, D684, U6912, E788, J9580]",361226::20180705,"[361121::20180705, 361122::20180705, 361C14::20180705, 3963::20180705, 3964::20180705, ...]",M,51,0,0,8,8,01,2018-07-04,0,00,2018-07-12


In [5]:
if unmatched.shape[0] > 0:
    unmatched

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-04 11:06:17.592 | INFO     | src.revised_case_normalization.py.group:group:12 - Grouping 17 cases ...
2022-11-04 11:06:19.396 | SUCCESS  | src.revised_case_normalization.py.group:group:16 - Grouped 17 cases into: 17 revisions, 168 diagnoses rows, 108 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,431229,F59B,2.55,0.97,4,2022-12-31
1,431363,F71A,1.12,1.12,4,2022-12-31
2,431860,F62B,1.30,1.30,4,2022-12-31
3,432553,F03C,4.90,4.90,4,2022-12-31
4,432647,F62B,1.30,1.30,4,2022-12-31
5,433607,F59B,2.55,1.50,4,2022-12-31
6,433746,L63B,1.09,1.09,4,2022-12-31
7,433916,L60C,1.40,1.40,4,2022-12-31
8,434365,I10A,2.66,2.66,4,2022-12-31
9,435248,F06C,3.84,3.84,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-04 11:06:19.417 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:224 - Trying to insert 17 cases into the 'Revisions' table ...
2022-11-04 11:06:19.947 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:250 - Deleted 17 rows from the "Revisions" table, which is about to be updated
2022-11-04 11:06:20.016 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:261 - Inserted 17 cases into the 'Revisions' table
2022-11-04 11:06:20.017 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:273 - Trying to insert 168 rows into the 'Diagnoses' table ...
2022-11-04 11:06:21.469 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:311 - Inserted 168 rows into the 'Diagnoses' table
2022-11-04 11:06:21.474 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:321 - Trying to insert 108 rows into the 'Procedures' ta